This notebook is only for testing. Maybe we can build a working model from this, and then we can start putting code into the "model" python module.

In [1]:
import sys
sys.path.append('../py')

In [2]:
import tensorflow as tf
import numpy as np
import time
import importlib

In [3]:
import cavitylearn.data
import cavitylearn.catalonet0

In [4]:
importlib.reload(cavitylearn.catalonet0)

<module 'cavitylearn.catalonet0' from '../py/cavitylearn/catalonet0.py'>

In [5]:
config = tf.ConfigProto(
        #device_count = {'GPU': 0}
    )

In [6]:
sess = tf.InteractiveSession(config=config)

In [7]:
with open('../../data/test/test-cavities.filenames.local.txt', 'rt') as cavfile:
    fnames = [r.strip() for r in cavfile]

In [8]:
dataconfig = cavitylearn.data.DataConfig(classes=["SO4","GOL","EDO","HEM","NAG","PO4","ACT"], num_props=4, boxshape=[30, 30, 30])
with open('../../data/test/labels.txt', 'rt') as labelfile:
    trainset = cavitylearn.data.DataSet(labelfile, '/var/cavitylearn', dataconfig)

In [9]:
trainset.N

21687

In [10]:
boxes_placeholder = tf.placeholder(tf.float32, shape=[None, dataconfig.boxshape[0], dataconfig.boxshape[1], dataconfig.boxshape[2], dataconfig.num_props])

In [11]:
labels_placholder = tf.placeholder(tf.float32, shape=[None, dataconfig.num_classes])

In [12]:
NUM_FILTERS=12
PIXELS= dataconfig.boxshape[0] * dataconfig.boxshape[1] * dataconfig.boxshape[2]
BATCHSIZE=100

In [13]:
with tf.variable_scope('conv1') as scope:
    kernel = tf.Variable(tf.truncated_normal([5,5,5,dataconfig.num_props,NUM_FILTERS]))
    conv = tf.nn.conv3d(boxes_placeholder, kernel, [1,1,1,1,1], padding='SAME')
    biases = tf.Variable(tf.zeros([NUM_FILTERS], dtype=tf.float32))
    bias = tf.nn.bias_add(conv, biases)
    conv1 = tf.nn.relu(bias, name=scope.name)

In [14]:
pool1 = tf.nn.max_pool3d(conv1, ksize=[1, 3, 3, 3, 1], strides=[1, 2, 2, 2, 1], padding='SAME')

In [15]:
norm1 = pool1 # tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta = 0.75, name='norm1')

In [20]:
logits = cavitylearn.catalonet0.inference(boxes_placeholder, dataconfig)

In [21]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(labels_placholder * tf.log(logits), reduction_indices=[1]))

In [22]:
sess.run(tf.initialize_all_variables())

In [23]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [24]:
correct_prediction = tf.equal(tf.argmax(logits,1),  tf.argmax(labels_placholder,1))

In [25]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [28]:
NUM_BATCHES=30

In [29]:
start_time = time.time()
for i in range(NUM_BATCHES):
    labels, boxes = trainset.next_batch(BATCHSIZE)
    train_step.run(feed_dict={boxes_placeholder: boxes, labels_placholder: labels})
    print(';', str(accuracy.eval(feed_dict={boxes_placeholder: boxes, labels_placholder: labels})), end='')
end_time = time.time()

; 0.13; 0.15; 0.17; 0.15; 0.22; 0.12; 0.18; 0.16; 0.19; 0.21; 0.16; 0.18; 0.2; 0.18; 0.2; 0.22; 0.17; 0.16; 0.16; 0.19; 0.18; 0.13; 0.14; 0.19; 0.13; 0.18; 0.13; 0.18; 0.14; 0.23

In [30]:
print("Total time: ", str(end_time - start_time))
print("Time per batch: ", str((end_time - start_time)/NUM_BATCHES))

Total time:  28.235718727111816
Time per batch:  0.9411906242370606
